In [22]:
import urllib2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json
from collections import defaultdict

import datetime as dt
import progressbar

import time

In [24]:


urls = {
    "Star Wars: Episode I The Phantom Menice": "Star_Wars:_Episode_I_The_Phantom_Menace",
    "Star Wars: Episode II Attack of the Clones": "Star_Wars:_Episode_II_Attack_of_the_Clones",
    "Star Wars: The Clone Wars (film)": "Star_Wars:_The_Clone_Wars_(film)",
    "Star Wars: Episode III Revenge of the Sith": "Star_Wars:_Episode_III_Revenge_of_the_Sith",
    "Solo: A Star Wars Story": "Solo:_A_Star_Wars_Story",
    "Rogue One: A Star Wars Story": "Rogue_One:_A_Star_Wars_Story",
    "Star Wars: Episode IV A New Hope": "Star_Wars:_Episode_IV_A_New_Hope",
    "Star Wars: Episode V The Empire Strikes Back": "Star_Wars:_Episode_V_The_Empire_Strikes_Back",
    "Star Wars: Episode VI Return of the Jedi": "Star_Wars:_Episode_VI_Return_of_the_Jedi",
    "Star Wars: Episode VII The Force Awakens": "Star_Wars:_Episode_VII_The_Force_Awakens",
    "Star Wars: Episode VIII The Last Jedi": "Star_Wars:_Episode_VIII_The_Last_Jedi"
}

partial_url = "http://starwars.wikia.com/api.php?action=query&prop=revisions&titles={0}&rvprop=content&format=json"

starwars_json = {"data": {"movies": defaultdict(dict), "series": defaultdict(dict), "characters": defaultdict(dict)},
               "failed": {"movies": {}, "characters": {}}}
for movie in urls.keys():
    query = partial_url.format(urls[movie])
    try:
        wikiresponse = urllib2.urlopen(query, timeout=2)
        wikisource = wikiresponse.read()

        # Load the response as .json and extract the relevant information into wikimarkup
        wikicontent = json.loads(wikisource)
        revision_nr = wikicontent["query"]["pages"].keys()[0]

        if revision_nr == "-1":
            starwars_json["failed"]["movies"][movie] = query
            print query
        else:
            wikimarkup = wikicontent["query"]["pages"][revision_nr]["revisions"][0]["*"]
            if ("l-characters=" in wikimarkup) and ("c-characters=" in wikimarkup):
                characters_text = wikimarkup.split("c-characters=")[-1].split("l-characters=")[0]
            elif "characters=" in wikimarkup:
                characters_text = wikimarkup.split("characters=")[-1].split("\n\n|")[0]
            else:
                print episode_query
            
            #TODO: use regex to find all links in characters_text and make a list that is stored in the dictionary
            character_names = list(set(re.findall("[\[]{2}(.+?)[\]]{2}", characters_text)))
            add_Vader = False
            for i in range(len(character_names)):
                if "|" in character_names[i]:
                    if character_names[i].split("|")[0] == "Anakin Skywalker": 
                        if "Darth Vader" in character_names[i].split("|")[-1] and "Anakin Skywalker" in character_names[i].split("|")[-1]:
                            character_names[i] = "Anakin Skywalker"
                            add_Vader = True
                        elif "Darth Vader" == character_names[i].split("|")[-1]:
                            print movie
                            character_names[i] = "Darth Vader"
                        else:
                            character_names[i] = "Anakin Skywalker"
                        
                    else:
                        character_names[i] = character_names[i].split("|")[0]
            if add_Vader:
                print movie
                character_names.append("Darth Vader")
                
            starwars_json["data"]["movies"][movie] = {"wikicontent": wikimarkup.encode('utf-8'), "characters": character_names}
            for n in character_names:
                starwars_json["data"]["characters"][n] = {}
            
    except Exception as e:
        print "Unexpected Failure During Movie Processing"
        print e
        print query

print "DONE PROCESSING MOVIES"
print        
print starwars_json["data"]["movies"].keys()

# with open("films.json", 'w') as f:
#     json.dumps(movies_json, f)

Star Wars: Episode III Revenge of the Sith
Star Wars: Episode V The Empire Strikes Back
Rogue One: A Star Wars Story
Star Wars: Episode IV A New Hope
Star Wars: Episode VIII The Last Jedi
Star Wars: Episode VI Return of the Jedi
DONE PROCESSING MOVIES

['Star Wars: Episode II Attack of the Clones', 'Star Wars: The Clone Wars (film)', 'Solo: A Star Wars Story', 'Star Wars: Episode VII The Force Awakens', 'Rogue One: A Star Wars Story', 'Star Wars: Episode VIII The Last Jedi', 'Star Wars: Episode V The Empire Strikes Back', 'Star Wars: Episode III Revenge of the Sith', 'Star Wars: Episode VI Return of the Jedi', 'Star Wars: Episode I The Phantom Menice', 'Star Wars: Episode IV A New Hope']


In [26]:
# animated_json = {"data": {"movies": defaultdict(dict), "series": defaultdict(dict), "characters": defaultdict(dict)},
#                "failed": {"movies": {}, "series": {}, "characters": {}}}

# animated_movies = ["Star Wars: The Clone Wars (film)"]

animated_series_names = ["Star Wars: The Clone Wars (series)", "Star Wars Rebels"]

animated_queries = {"Star Wars: The Clone Wars (series)": "http://starwars.wikia.com/api.php?action=query&list=categorymembers&cmtitle=Category:The_Clone_Wars_episodes&cmlimit=500&format=json",
                   "Star Wars Rebels": "http://starwars.wikia.com/api.php?action=query&list=categorymembers&cmtitle=Category:Star_Wars_Rebels_episodes&cmlimit=500&format=json"}
print "STARTING TO PROCESS SERIES AT {0}".format(dt.datetime.now())
failed_episodes = defaultdict(dict)
for series in animated_series_names:
    series_query = animated_queries[series]
    series_response = urllib2.urlopen(series_query)
    series_source = series_response.read()
    series_content = json.loads(series_source)
    
    episode_list = [x["title"] for x in series_content["query"]["categorymembers"]]
    print series

    for episode in episode_list:
        if not episode.startswith("Category:"):
            r = 0
            while True:
                try:
                    episode_query = partial_url.format(episode.replace(' ', '_').encode('utf-8'))
                    episode_response = urllib2.urlopen(episode_query, timeout=2.0)
                    episode_source = episode_response.read()
                    episode_content = json.loads(episode_source)

                    revision_nr = episode_content["query"]["pages"].keys()[0]

                    if revision_nr == "-1":
                        starwars_json["failed"]["series"][episode] = episode_query
                        print episode_query
                    else:
                        episode_markup = episode_content["query"]["pages"][revision_nr]["revisions"][0]["*"]
                        if ("l-characters=" in episode_markup) and ("c-characters=" in episode_markup):
                            characters_text = episode_markup.split("c-characters=")[-1].split("l-characters=")[0]

                        elif "characters=" in episode_markup:
                            characters_text = episode_markup.split("characters=")[-1].split("\n\n|")[0]
                        else:
                            characters_text = ""
                            print episode_query

                        #TODO: use regex to find all links in characters_text and make a list that is stored in the dictionary
                        character_names = list(set(re.findall("[\[]{2}(.+?)[\]]{2}", characters_text)))
                        add_Vader = False
                        for i in range(len(character_names)):
                            if "|" in character_names[i]:
                                if character_names[i].split("|")[0] == "Anakin Skywalker": 
                                    if "Darth Vader" in character_names[i].split("|")[-1] and "Anakin Skywalker" in character_names[i].split("|")[-1]:
                                        character_names[i] = "Anakin Skywalker"
                                        add_Vader = True
                                    elif "Darth Vader" == character_names[i].split("|")[-1]:
                                        character_names[i] = "Darth Vader"
                                        print series, episode
                                    else:
                                        character_names[i] = "Anakin Skywalker"
                                else:
                                    character_names[i] = character_names[i].split("|")[0]
                        if add_Vader:
                            character_names.append("Darth Vader")
                            print series, episode
                            
                        starwars_json["data"]["series"][series][episode] = {"wikicontent": episode_markup.encode('utf-8'), "characters": character_names}
                        for n in character_names:
                            starwars_json["data"]["characters"][n] = {}            
                    break

                except Exception as e:
                    r += 1
                    time.sleep(5)
                    if r > 20:
                        print "Unexpected Failure During Episode Processing"
                        print e
                        print episode_query
                        failed_episodes[series][episode] = episode_query
                        break
            
                finally:
                    time.sleep(0.5)
        
print starwars_json["data"]["series"].keys()
print failed_episodes

STARTING TO PROCESS SERIES AT 2018-11-15 12:39:45.561000
Star Wars: The Clone Wars (series)
Star Wars: The Clone Wars (series) Ghosts of Mortis
Star Wars Rebels
Star Wars Rebels Star Wars Rebels: Spark of Rebellion
Star Wars Rebels Gathering Forces
Star Wars Rebels Fire Across the Galaxy
Star Wars Rebels Star Wars Rebels: The Siege of Lothal
Star Wars Rebels The Lost Commanders
Star Wars Rebels Relics of the Old Republic
Star Wars Rebels Always Two There Are
Star Wars Rebels The Future of the Force
Star Wars Rebels Twilight of the Apprentice
Star Wars Rebels Star Wars Rebels: Steps Into Shadow
Star Wars Rebels Twin Suns (episode)
Star Wars Rebels A World Between Worlds
['Star Wars Rebels', 'Star Wars: The Clone Wars (series)']
defaultdict(<type 'dict'>, {})


In [28]:
print "STARTING TO PROCESS CHARACTERS AT {0}".format(dt.datetime.now())
print len(starwars_json["data"]["characters"].keys())
all_affiliations = set([])
try_again_list = []
for n in starwars_json["data"]["characters"].keys():
    r = 0
    while True:
        try:
            if n == "Darth Vader":
                character_query = partial_url.format("Anakin_Skywalker")
            else:
                character_query = partial_url.format(n.replace(' ', '_').encode('utf-8'))
            character_response = urllib2.urlopen(character_query, timeout=2)
            character_source = character_response.read()
            character_content = json.loads(character_source)

            revision_nr = character_content["query"]["pages"].keys()[0]

            if revision_nr == "-1":
                starwars_json["failed"]["characters"][n] = character_query
                print character_query
            else:
                character_markup = character_content["query"]["pages"][revision_nr]["revisions"][0]["*"]

                links = list(set(re.findall("[\[]{2}(.+?)[\]]{2}", character_markup)))
                add_Vader = False
                for i in range(len(links)):
                    if links[i].split("|")[0] == "Anakin Skywalker": 
                        if "Darth Vader" in links[i].split("|")[-1] and "Anakin Skywalker" in links[i].split("|")[-1]:
                            links[i] = "Anakin Skywalker"
                            add_Vader = True
                        elif "Darth Vader" == links[i].split("|")[-1]:
                            links[i] = "Darth Vader"
                            print n
                        else:
                            links[i] = "Anakin Skywalker"
                    else:
                        links[i] = links[i].split("|")[0]
                if add_Vader:
                    links.append("Darth Vader")
                    print n

                links = set(links)

                character_links = list(links.intersection(set(starwars_json["data"]["characters"].keys())))
                if "Anakin Skywalker" in character_links and "Darth Vader" not in character_links:
                    if "Darth Vader" in character_markup:
                        character_links.append("Darth Vader")
                
                if n == "Anakin Skywalker" and "Darth Vader" not in character_links:
                    character_links.append("Darth Vader")
                if n == "Darth Vader" and "Anakin Skywalker" not in character_links:
                    character_links.append("Anakin Skywalker")

                affiliations_text = ""
                if "|affiliation=" in character_markup:
                    affiliations_text = character_markup.split("|affiliation")[1].split("\n|")[0].split("}\n")[0]

                if len(affiliations_text) >= 3:
                    if affiliations_text[1:3] == "}}":
                        affiliations_text = ""

                if len(affiliations_text) > 1:
                    if affiliations_text[1] == "*":
                        affiliations = re.findall("[\n=]\*[\[]{2}(.+?)[\]]{2}", affiliations_text)
                    else:
                        affiliations = re.findall("[\n=][\[]{2}(.+?)[\]]{2}", affiliations_text)
                else:
                    affiliations = []

                for i in range(len(affiliations)):
                    affiliations[i] = affiliations[i].split("|")[0]

                starwars_json["data"]["characters"][n] = {"wikicontent": character_markup,
                                                          "connections": character_links,
                                                          "affiliations": affiliations}
                all_affiliations = all_affiliations.union(affiliations)
                
            break

        except Exception as e:
            r += 1
            time.sleep(5)
            if r > 20:
                print "Unexpected Failure During Character Processing"
                print e
                print character_query
                try_again_list.append(n)
                

        finally:
            time.sleep(0.5)
        
print "DONE"
print len(all_affiliations)
print try_again_list

STARTING TO PROCESS CHARACTERS AT 2018-11-15 12:43:26.676000
1978
Firmus Piett
4-LOM
Presence
Sim Aloo
Tiaan Jerjerrod
CC-1010
Nippet
http://starwars.wikia.com/api.php?action=query&prop=revisions&titles=Vamasto_Maja&rvprop=content&format=json
2-1B
Dex Tiree
http://starwars.wikia.com/api.php?action=query&prop=revisions&titles=NP-G1&rvprop=content&format=json
CR978
Orson Callan Krennic
Edmos Khurgee
Seventh Sister
Cat Miin
JA189
Puck Naeco
Unidentified Imperial Navy Trooper
Boba Fett
Unidentified Stormtrooper Commander (Imperial Complex)
Garven Dreis
Phasma
Maximillian Veers
Eeth Koth
J'Quille
Han Solo
Hija
http://starwars.wikia.com/api.php?action=query&prop=revisions&titles=Unidentified_Mandalorian_Minister_of_the_Interior&rvprop=content&format=json
Eighth Brother
Gherant
Dengar
http://starwars.wikia.com/api.php?action=query&prop=revisions&titles=Dezine_Krisso&rvprop=content&format=json
Karina the Great (Age of the Empire)
Passel Argente
Vult Skerris
Cin Drallig
Rey
Lajaie
Jabba Desilij

In [30]:
with open("starwars_vader_split.json", 'w') as f:
    json.dump(starwars_json, f)

In [7]:
with open("starwars.json", 'r') as f:
    json_new = json.load(f)

with open("starwars-Copy1.json", 'r') as f:
    json_old = json.load(f)
    
print len(json_new["data"]["characters"])
print len(json_old["data"]["characters"])

print set(json_new["data"]["characters"]).difference(set(json_old["data"]["characters"]))

for n in set(json_new["data"]["characters"]).difference(set(json_old["data"]["characters"])):
    print json_new["data"]["characters"][n]

1276
1977
set([u'Darth Vader'])
{u'wikicontent': u'#redirect [[Anakin Skywalker]] {{R from alternate name}}', u'connections': [u'Anakin Skywalker'], u'affiliations': []}
